# California Housing Prices - Feedforward Neural Network

Predict the median house price using the other features

In [278]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
 


In [281]:
df = pd.read_csv('housing.csv')

In [279]:
X = df.drop(columns='median_house_value')
y = df['median_house_value'].to_numpy()

#need to cgange ocean proximity to binary codes
X = pd.get_dummies(X, columns=['ocean_proximity'], drop_first=True).to_numpy()



In [282]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Scale and split data

In [280]:
X_scaler = StandardScaler()
y_scaler = StandardScaler()
X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1,1)).flatten() #reshape to 2d array then flatten to 1D vector

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Convert to tensor

In [284]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device).unsqueeze(1)


# Create Model

In [283]:
class HousePriceRegression(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(HousePriceRegression, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x
    
    
    
    
input_size = X.shape[1]
hidden_size = 64
output_size = 1 #one continueous value no need to classify


print(f'Input Size: {input_size}, Output Size: {output_size}')

Input Size: 12, Output Size: 1


In [285]:
# create instance of model
model = HousePriceRegression(input_size, hidden_size, output_size).to(device)

criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr = 0.001)

In [286]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()

    y_predicted = model(X_train_tensor)
    loss = criterion(y_predicted, y_train_tensor)

    #back propogation
    loss.backward()
    optimiser.step()
    optimiser.zero_grad()

    if(epoch+1)%100 == 0:
        print(f'epoch {epoch+1}, loss: {loss.item():.4f}')

epoch 100, loss: 0.8574


# Evaluate Model

In [287]:
model.eval()
with torch.no_grad():
    y_predicted = model(X_test_tensor).cpu().numpy()
    y_test = y_test_tensor.cpu().numpy()

    # Inverse transform to get original scale
    y_predicted = y_scaler.inverse_transform(y_predicted)
    y_test = y_scaler.inverse_transform(y_test)

    # Check for NaNs
    if np.isnan(y_predicted).any() or np.isnan(y_test).any():
        print("NaNs found in predictions or true values.")

    mse = mean_squared_error(y_test, y_predicted)
    mae = mean_absolute_error(y_test, y_predicted)
    r2 = r2_score(y_test, y_predicted)

    print(f'Test MSE: {mse:.4f}')
    print(f'Test MAE: {mae:.4f}')
    print(f'Test R²: {r2:.4f}')

NaNs found in predictions or true values.


ValueError: Input contains NaN.